# Lecture 2 - Fourier reconstruction

# Contents

* The Fourier-Slice Theorem
* Fourier reconstruction

# The Fourier-Slice Theorem

The Fourier-Slice Theorem relates the 2D-Fourier transform of $u$ to the 1D-Fourier transform of $f$:

$$\widehat{f}(\sigma,\theta) = \widehat{u}(|\sigma| \mathbf{n}_\theta),$$

with

$$\widehat{f}(\sigma,\theta) = \int_{\mathbb{R}} f(s,\theta) e^{-2\pi\imath \sigma s}\mathrm{d}s,$$

$$\widehat{u}(\mathbf{k}) = \int_{\mathbb{R}^2} u(\mathbf{x}) e^{-2\pi\imath \mathbf{k}\cdot \mathbf{x}}\mathrm{d}\mathbf{x},$$

$$\mathbf{n}_\theta = (\cos\theta,\sin\theta).$$

We can visualise this as follows

![](https://upload.wikimedia.org/wikipedia/commons/thumb/9/97/Radon_transform_via_Fourier_transform.png/2000px-Radon_transform_via_Fourier_transform.png)

# Fourier reconstruction

* In principle, we can reconstruct the image by performing an inverse Fourier transform
* Having collected *discrete* measurements $f_{ij} = f(s_i,\theta_j)$, we only have partial information on the Fourier transform of $u$

## The Discrete Fourier Transform

The *discrete Fourier transform (DFT)* of a sequence $\{g_i\}_{i=0}^n$ is defined as

$$\widehat{g}_i = \sum_{j=0}^{n-1} g_j \exp\left(-\frac{2\pi\imath}{n} ij\right), \quad i = 0, \ldots, n-1.$$

If $g_i = g(i\cdot\Delta x)$, then $\widehat{g}_i = \widehat{g}( \ldots )$.

In matrix-vector notation, we express this as

$$\widehat{\mathbf{g}} = F\mathbf{g},$$

with $F_{ij} = \exp\left(-\frac{2\pi\imath}{n} ij\right)$.

The inverse is given by

$$F^{-1} = n^{-1} F^*.$$

## The Fast Fourier Transform

A naive implementation would require $\mathcal{O}(n^2)$ operations to multiply by $F$. With some clever tricks, however, it can be done in $\mathcal{O}(n\log n)$.

## Interpolation

* Applying the Fourier transform to $f_{ij}$ gives us samples of $\widehat{u}$ at wavenumbers $\{(\sigma_i \mathbf{n}_{\theta_j})\}$, with $\sigma_i = \ldots$ (assuming again that $f_{ij} = f(i \cdot \Delta s ,
\theta_j)$)
* To usefully apply the inverse DFT, we need $\widehat{u}$ at wavenumbers $\{(\ldots, \ldots)\}$.

A linear interpolation scheme ...

Representing the sinogram and image as vectors, the image recontruction step can be represented as 

$$\mathbf{u} = (F^{-1} \otimes F^{-1}) T (I \otimes F) \mathbf{f}.$$

In Python, we would implement this as 

```python
u .. 
```

## The Non-Uniform Fourier Transform

We can avoid the interpolation step by applying a *non-uniform FFT*.